# Lab 12: Policy and Claims APIs

## Learning Objectives
- Implement policy administration endpoints
- Create claims processing APIs
- Build policy search and retrieval functionality
- Handle claims creation and status updates

**Duration:** 45 minutes  
**Prerequisites:** Completion of Customer Management APIs

## Step 1: Policy Management Endpoints

Implement comprehensive policy administration with creation, retrieval, and listing.

In [ ]:
# Cell 8: Policy management API endpoints
@app.post("/policies", response_model=PolicyResponse, tags=["Policy Management"])
async def create_policy(
    policy_data: PolicyCreate,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """Create new insurance policy"""
    
    # Verify customer exists
    customer_check = """
    MATCH (c:Customer {customer_id: $customer_id})
    RETURN c.customer_id as customer_id, 
           c.first_name + ' ' + c.last_name as customer_name
    """
    
    customer_result = connection_manager.execute_query(
        customer_check, 
        {"customer_id": policy_data.customer_id}
    )
    
    if not customer_result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Customer not found"
        )
    
    customer_name = customer_result[0]["customer_name"]
    
    # Generate policy ID and number
    policy_id = f"POL_{secrets.token_hex(6).upper()}"
    policy_number = f"INS-{datetime.now().year}-{secrets.token_hex(4).upper()}"
    
    # Calculate dates
    start_date = date.today()
    end_date = start_date + timedelta(days=30 * policy_data.policy_term_months)
    
    # Create policy
    create_query = """
    MATCH (c:Customer {customer_id: $customer_id})
    CREATE (p:Policy {
        policy_id: $policy_id,
        policy_number: $policy_number,
        product_name: $product_name,
        status: 'Active',
        coverage_amount: $coverage_amount,
        premium_amount: $premium_amount,
        deductible: $deductible,
        policy_term_months: $policy_term_months,
        start_date: date($start_date),
        end_date: date($end_date),
        created_date: datetime(),
        created_by: $created_by
    })
    CREATE (c)-[:HAS_POLICY]->(p)
    RETURN p
    """
    
    policy_dict = policy_data.dict()
    policy_dict.update({
        "policy_id": policy_id,
        "policy_number": policy_number,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "created_by": current_user["sub"]
    })
    
    result = connection_manager.execute_write_query(create_query, policy_dict)
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail="Failed to create policy"
        )
    
    # Return created policy
    return PolicyResponse(
        policy_id=policy_id,
        policy_number=policy_number,
        customer_id=policy_data.customer_id,
        customer_name=customer_name,
        product_name=policy_data.product_name,
        status=PolicyStatus.ACTIVE,
        coverage_amount=policy_data.coverage_amount,
        premium_amount=policy_data.premium_amount,
        deductible=policy_data.deductible,
        policy_term_months=policy_data.policy_term_months,
        start_date=start_date,
        end_date=end_date,
        created_date=datetime.now()
    )

@app.get("/policies/{policy_id}", response_model=PolicyResponse, tags=["Policy Management"])
async def get_policy(
    policy_id: str,
    current_user: Dict[str, Any] = Depends(get_current_user)
):
    """Get policy by ID"""
    
    query = """
    MATCH (c:Customer)-[:HAS_POLICY]->(p:Policy {policy_id: $policy_id})
    RETURN p.policy_id as policy_id,
           p.policy_number as policy_number,
           c.customer_id as customer_id,
           c.first_name + ' ' + c.last_name as customer_name,
           p.product_name as product_name,
           p.status as status,
           p.coverage_amount as coverage_amount,
           p.premium_amount as premium_amount,
           p.deductible as deductible,
           p.policy_term_months as policy_term_months,
           p.start_date as start_date,
           p.end_date as end_date,
           p.created_date as created_date
    """
    
    result = connection_manager.execute_query(query, {"policy_id": policy_id})
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Policy not found"
        )
    
    return PolicyResponse(**result[0])

print("✓ Policy creation and retrieval endpoints configured")

## Step 2: Policy Listing with Filtering

Implement policy listing with status filtering and customer-based queries.

In [ ]:
@app.get("/policies", response_model=PaginatedResponse, tags=["Policy Management"])
async def list_policies(
    page: int = 1,
    per_page: int = 10,
    status: Optional[PolicyStatus] = None,
    customer_id: Optional[str] = None,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """List policies with pagination and filtering"""
    
    # Build WHERE clause
    where_conditions = []
    params = {}
    
    if status:
        where_conditions.append("p.status = $status")
        params["status"] = status.value
    
    if customer_id:
        where_conditions.append("c.customer_id = $customer_id")
        params["customer_id"] = customer_id
    
    where_clause = "WHERE " + " AND ".join(where_conditions) if where_conditions else ""
    
    # Count total policies
    count_query = f"""
    MATCH (c:Customer)-[:HAS_POLICY]->(p:Policy)
    {where_clause}
    RETURN count(p) as total
    """
    
    total_result = connection_manager.execute_query(count_query, params)
    total = total_result[0]["total"]
    
    # Get paginated policies
    skip = (page - 1) * per_page
    params.update({"skip": skip, "limit": per_page})
    
    list_query = f"""
    MATCH (c:Customer)-[:HAS_POLICY]->(p:Policy)
    {where_clause}
    RETURN p.policy_id as policy_id,
           p.policy_number as policy_number,
           c.customer_id as customer_id,
           c.first_name + ' ' + c.last_name as customer_name,
           p.product_name as product_name,
           p.status as status,
           p.coverage_amount as coverage_amount,
           p.premium_amount as premium_amount,
           p.deductible as deductible,
           p.policy_term_months as policy_term_months,
           p.start_date as start_date,
           p.end_date as end_date,
           p.created_date as created_date
    ORDER BY p.created_date DESC
    SKIP $skip LIMIT $limit
    """
    
    result = connection_manager.execute_query(list_query, params)
    policies = [PolicyResponse(**record) for record in result]
    
    return PaginatedResponse(
        items=policies,
        total=total,
        page=page,
        per_page=per_page,
        pages=(total + per_page - 1) // per_page
    )

print("✓ Policy management API endpoints configured")

## Step 3: Claims Processing Endpoints

Implement claims submission, retrieval, and status update functionality.

In [ ]:
# Cell 9: Claims processing API endpoints
@app.post("/claims", response_model=ClaimResponse, tags=["Claims Processing"])
async def create_claim(
    claim_data: ClaimCreate,
    current_user: Dict[str, Any] = Depends(get_current_user)
):
    """Submit new insurance claim"""
    
    # Verify policy exists and is active
    policy_check = """
    MATCH (c:Customer)-[:HAS_POLICY]->(p:Policy {policy_id: $policy_id})
    WHERE p.status = 'Active' AND p.start_date <= date() AND p.end_date >= date()
    RETURN p.policy_id as policy_id,
           p.policy_number as policy_number,
           c.first_name + ' ' + c.last_name as customer_name,
           p.coverage_amount as coverage_amount,
           p.deductible as deductible
    """
    
    policy_result = connection_manager.execute_query(
        policy_check,
        {"policy_id": claim_data.policy_id}
    )
    
    if not policy_result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Active policy not found"
        )
    
    policy_info = policy_result[0]
    
    # Validate claim amount against coverage
    if claim_data.claim_amount > policy_info["coverage_amount"]:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=f"Claim amount exceeds policy coverage limit of ${policy_info['coverage_amount']:,.2f}"
        )
    
    # Generate claim ID and number
    claim_id = f"CLM_{secrets.token_hex(6).upper()}"
    claim_number = f"CLM-{datetime.now().year}-{secrets.token_hex(4).upper()}"
    
    # Create claim
    create_query = """
    MATCH (p:Policy {policy_id: $policy_id})
    CREATE (cl:Claim {
        claim_id: $claim_id,
        claim_number: $claim_number,
        status: 'Submitted',
        claim_amount: $claim_amount,
        incident_date: date($incident_date),
        filed_date: datetime(),
        description: $description,
        incident_type: $incident_type,
        location: $location,
        filed_by: $filed_by
    })
    CREATE (p)-[:HAS_CLAIM]->(cl)
    RETURN cl
    """
    
    claim_dict = claim_data.dict()
    claim_dict.update({
        "claim_id": claim_id,
        "claim_number": claim_number,
        "incident_date": claim_data.incident_date.isoformat(),
        "filed_by": current_user["sub"]
    })
    
    result = connection_manager.execute_write_query(create_query, claim_dict)
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_500_INTERNAL_SERVER_ERROR,
            detail="Failed to create claim"
        )
    
    # Return created claim
    return ClaimResponse(
        claim_id=claim_id,
        claim_number=claim_number,
        policy_id=claim_data.policy_id,
        policy_number=policy_info["policy_number"],
        customer_name=policy_info["customer_name"],
        status=ClaimStatus.SUBMITTED,
        claim_amount=claim_data.claim_amount,
        incident_date=claim_data.incident_date,
        filed_date=datetime.now(),
        description=claim_data.description,
        incident_type=claim_data.incident_type,
        location=claim_data.location,
        adjuster_name=None
    )

@app.get("/claims/{claim_id}", response_model=ClaimResponse, tags=["Claims Processing"])
async def get_claim(
    claim_id: str,
    current_user: Dict[str, Any] = Depends(get_current_user)
):
    """Get claim by ID"""
    
    query = """
    MATCH (c:Customer)-[:HAS_POLICY]->(p:Policy)-[:HAS_CLAIM]->(cl:Claim {claim_id: $claim_id})
    OPTIONAL MATCH (cl)-[:ASSIGNED_TO]->(adj:Agent)
    RETURN cl.claim_id as claim_id,
           cl.claim_number as claim_number,
           p.policy_id as policy_id,
           p.policy_number as policy_number,
           c.first_name + ' ' + c.last_name as customer_name,
           cl.status as status,
           cl.claim_amount as claim_amount,
           cl.incident_date as incident_date,
           cl.filed_date as filed_date,
           cl.description as description,
           cl.incident_type as incident_type,
           cl.location as location,
           adj.first_name + ' ' + adj.last_name as adjuster_name
    """
    
    result = connection_manager.execute_query(query, {"claim_id": claim_id})
    
    if not result:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Claim not found"
        )
    
    return ClaimResponse(**result[0])

@app.put("/claims/{claim_id}/status", response_model=ClaimResponse, tags=["Claims Processing"])
async def update_claim_status(
    claim_id: str,
    status_update: ClaimUpdate,
    current_user: Dict[str, Any] = Depends(require_role(UserRole.ADJUSTER))
):
    """Update claim status (adjusters only)"""
    
    # Check if claim exists
    check_query = """
    MATCH (cl:Claim {claim_id: $claim_id})
    RETURN cl.claim_id as claim_id
    """
    
    existing = connection_manager.execute_query(check_query, {"claim_id": claim_id})
    if not existing:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail="Claim not found"
        )
    
    # Update claim
    update_fields = {k: v for k, v in status_update.dict().items() if v is not None}
    
    if not update_fields:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="No valid fields to update"
        )
    
    # Handle status updates
    if "status" in update_fields:
        update_fields["status"] = update_fields["status"].value
    
    set_clauses = [f"cl.{field} = ${field}" for field in update_fields.keys()]
    set_clause = "SET " + ", ".join(set_clauses)
    set_clause += ", cl.last_updated = datetime(), cl.updated_by = $updated_by"
    
    update_fields.update({
        "claim_id": claim_id,
        "updated_by": current_user["sub"]
    })
    
    update_query = f"""
    MATCH (cl:Claim {{claim_id: $claim_id}})
    {set_clause}
    RETURN cl
    """
    
    connection_manager.execute_write_query(update_query, update_fields)
    
    # Return updated claim
    return await get_claim(claim_id, current_user)

print("✓ Claims processing API endpoints configured")